#### Tải bộ dữ liệu

In [1]:
import os
import random
import time
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from torchsummary import summary


import matplotlib.pyplot as plt
from PIL import Image

ROOT = './data'
train_data = datasets.MNIST(
    root=ROOT,
    train=True,
    download=True
)
test_data = datasets.MNIST(
    root=ROOT,
    train=False,
    download=True
)

#### Tiền xử lý dữ liệu

In [2]:
# split training: validation = 0.9:0.1
VALID_RARIO = 0.9

n_train_examples = int(len(train_data) * VALID_RARIO)
n_valid_examples = len(train_data) - n_train_examples

train_data, valid_data = data.random_split(
    train_data,
    [n_train_examples, n_valid_examples]
)

# compute mean and std for normalization
mean = train_data.dataset.data.float().mean() / 255
std = train_data.dataset.data.float().std() / 255

train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[mean], std=[std])
])
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[mean], std=[std])
])

train_data.dataset.transform = train_transforms
valid_data.dataset.transform = test_transforms

# create dataloader
BATCH_SIZE = 256

train_dataloader = data.DataLoader(
    train_data,
    shuffle=True,
    batch_size=BATCH_SIZE,
    num_workers=0
)
valid_dataloader = data.DataLoader(
    valid_data,
    batch_size=BATCH_SIZE,
    num_workers=0
)

#### Xây dựng mô hình LeNet

In [3]:
class LeNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=1,
            out_channels=6,
            kernel_size=5,
            padding='same'
        )
        self.avgpool1 = nn.AvgPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(
            in_channels=6,
            out_channels=16,
            kernel_size=5
        )
        self.avgpool2 = nn.AvgPool2d(kernel_size=2)
        self.flatten = nn.Flatten()
        self.fc_1 = nn.Linear(16 * 5 * 5, 120)
        self.fc_2 = nn.Linear(120, 84)
        self.fc_3 = nn.Linear(84, num_classes)

    def forward(self, inputs):
        outputs = self.conv1(inputs)
        outputs = self.avgpool1(outputs)
        outputs = F.relu(outputs)
        outputs = self.conv2(outputs)
        outputs = self.avgpool2(outputs)
        outputs = F.relu(outputs)
        outputs = self.flatten(outputs)
        outputs = self.fc_1(outputs)
        outputs = self.fc_2(outputs)
        outputs = self.fc_3(outputs)
        return outputs

#### Huấn luyện mô hình

In [4]:
def train(model, optimizer, criterion, train_dataloader, device, epoch=0, log_interval=50):
    model.train()
    total_acc, total_count = 0, 0
    losses = []
    start_time = time.time()

    for idx, (inputs, labels) in enumerate(train_dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        predictions = model(inputs)

        # compute loss
        loss = criterion(predictions, labels)
        losses.append(loss.item())

        # backward
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predictions.argmax(1) == labels).sum().item()
        total_count += labels.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches'
                  '| accuracy {:8.3f}'.format(
                      epoch, idx, len(train_dataloader), total_acc / total_count
                  ))
            total_acc, total_count = 0, 0
            start_time = time.time()
    epoch_acc = total_acc / total_count
    epoch_loss = sum(losses) / len(losses)
    return epoch_acc, epoch_loss


# Evaludation funcction
def evaluate(model, criterion, valid_dataloader, device):
    model.eval()
    total_acc, total_count = 0, 0
    losses = []

    with torch.no_grad():
        for idx, (inputs, lables) in enumerate(valid_dataloader):
            inputs = inputs.to(device)
            labels = lables.to(device)

            predictions = model(inputs)

            loss = criterion(predictions, labels)
            losses.append(loss.item())

            total_acc += (predictions.argmax(1) == labels).sum().item()
            total_count += labels.size(0)

    epoch_acc = total_acc / total_count
    epoch_loss = sum(losses) / len(losses)
    return epoch_acc, epoch_loss

In [5]:
num_classes = len(train_data.dataset.classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

lenet_model = LeNetClassifier(num_classes)
lenet_model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(lenet_model.parameters(), lr=0.001, weight_decay=0)

num_epochs = 10
save_model = '../weights'

train_accs, train_losses = [], []
eval_accs, eval_losses = [], []
best_loss_eval = 100

for epoch in range(1, num_epochs+1):
    epoch_start_time = time.time()
    # Training
    train_acc, train_loss = train(model=lenet_model,
                                  optimizer=optimizer,
                                  criterion=criterion,
                                  train_dataloader=train_dataloader,
                                  device=device,
                                  epoch=epoch)
    train_accs.append(train_acc)
    train_losses.append(train_loss)

    # Evaluation
    eval_acc, eval_loss = evaluate(model=lenet_model,
                                   criterion=criterion,
                                   valid_dataloader=valid_dataloader,
                                   device=device)
    eval_accs.append(eval_acc)
    eval_losses.append(eval_loss)

    # Save best model
    if eval_loss < best_loss_eval:
        torch.save(lenet_model.state_dict(), save_model + '/lenet_model_[mnist_dataset].pt')

    # Print loss, acc end epoch
    print('-'* 59)
    print(
        '| End of epoch {:3d} | Time: {:5.2f}s | Train Accuracy {:8.3f} | Train Loss {:8.3f} '
        '| Valid Accuracy {:8.3f} | Valid Loss {:8.3f} '.format(epoch, time.time() - epoch_start_time, train_acc, train_loss, eval_acc, eval_loss)
    )
    print('-'* 59)

    # Load best model
    lenet_model.load_state_dict(torch.load(save_model + '/lenet_model_[mnist_dataset].pt'))
    lenet_model.eval()

| epoch   1 |    50/  211 batches| accuracy    0.690
| epoch   1 |   100/  211 batches| accuracy    0.882
| epoch   1 |   150/  211 batches| accuracy    0.912
| epoch   1 |   200/  211 batches| accuracy    0.937
-----------------------------------------------------------
| End of epoch   1 | Time: 20.43s | Train Accuracy    0.949 | Train Loss    0.488 | Valid Accuracy    0.948 | Valid Loss    0.182 
-----------------------------------------------------------


C:\Users\FPTSHOP\AppData\Local\Temp\ipykernel_1968\3088716186.py:50: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lenet_model.load_state_dict(torch.load(save_model + '/lene

| epoch   2 |    50/  211 batches| accuracy    0.954
| epoch   2 |   100/  211 batches| accuracy    0.963
| epoch   2 |   150/  211 batches| accuracy    0.964
| epoch   2 |   200/  211 batches| accuracy    0.969
-----------------------------------------------------------
| End of epoch   2 | Time: 16.27s | Train Accuracy    0.967 | Train Loss    0.126 | Valid Accuracy    0.968 | Valid Loss    0.107 
-----------------------------------------------------------
| epoch   3 |    50/  211 batches| accuracy    0.970
| epoch   3 |   100/  211 batches| accuracy    0.973
| epoch   3 |   150/  211 batches| accuracy    0.973
| epoch   3 |   200/  211 batches| accuracy    0.974
-----------------------------------------------------------
| End of epoch   3 | Time: 15.47s | Train Accuracy    0.970 | Train Loss    0.090 | Valid Accuracy    0.962 | Valid Loss    0.116 
-----------------------------------------------------------
| epoch   4 |    50/  211 batches| accuracy    0.975
| epoch   4 |   100/ 

#### Đánh giá mô hình trên tập test

In [6]:
test_data.transform = test_transforms
test_dataloader = data.DataLoader(
    test_data,
    batch_size=BATCH_SIZE,
    num_workers=0
)
test_acc, test_loss = evaluate(lenet_model, criterion, test_dataloader, device)
test_acc, test_loss

(0.9859, 0.04435783763019572)